<div style='background-image: url("../share/images/header.svg") ; padding: 0px ; background-size: cover ; border-radius: 5px ; height: 250px'>
    <div style="float: right ; margin: 50px ; padding: 20px ; background: rgba(255 , 255 , 255 , 0.7) ; width: 50% ; height: 150px">
        <div style="position: relative ; top: 50% ; transform: translatey(-50%)">
            <div style="font-size: xx-large ; font-weight: 900 ; color: rgba(0 , 0 , 0 , 0.8) ; line-height: 100%">ObsPy Tutorial</div>
            <div style="font-size: large ; padding-top: 20px ; color: rgba(0 , 0 , 0 , 0.5)">Downloading Data</div>
        </div>
    </div>
</div>

Seismo-Live: http://seismo-live.org

##### Authors:
* Lion Krischer ([@krischer](https://github.com/krischer))
* Tobias Megies ([@megies](https://github.com/megies))
* Yannik Behr ([@yannikbehr](https://github.com/yannikbehr))
---

![](https://github.com/GNS-Science/BMKG_OBSPY_WORKSHOP/blob/main/images/obspy_logo_full_524x179px.png?raw=1)

In [ ]:
try:
  import obspy
  import cartopy
except ModuleNotFoundError:
  !pip -qq install obspy cartopy

In [ ]:
import matplotlib.pyplot as plt
plt.style.use('ggplot')
plt.rcParams['figure.figsize'] = 12, 8

ObsPy has clients to directly fetch data via...

- FDSN webservices (IRIS, Geofon/GFZ, USGS, NCEDC, SeisComp3 instances, ...)
- ArcLink (EIDA, ...)
- Earthworm
- SeedLink (near-realtime servers)
- NERIES/NERA/seismicportal.eu
- NEIC
- SeisHub (local seismological database)

This introduction shows how to use the FDSN webservice client. The FDSN webservice definition is by now the default web service implemented by many data centers world wide. Clients for other protocols work similar to the FDSN client.

#### Waveform Data

Let's start by finding some interesting stations. We will have a look in Iris' massive waveform archive for stations around Krakatao. To do so you can use their [Google Map Service](https://ds.iris.edu/gmap). [This example](https://ds.iris.edu/gmap/#starttime=2024-12-09&maxlat=6.2353&maxlon=141.233&minlat=-12.9331&minlon=91.7506&network=*&drawingmode=box&networktype=permanent&planet=earth) shows one particular search.

We will look at 4 stations:

* GE.BBJI..LHZ
* GE.MNAI..BHZ
* GE.PMBI..BHZ
* GE.SMRI..BHZ
* GE.UGM..BHZ

We retrieved the date of the tsunamigenic eruption on 22 December 2018 from the [Global Volcanism Program](https://volcano.si.edu/volcano.cfm?vn=262000)

In [ ]:
import os
from obspy import UTCDateTime, Stream, read
from obspy.clients.fdsn import Client

fout = 'anak_krakatau_2018.mseed'
if not os.path.isfile(fout):
    client = Client("Geofon")
    t = UTCDateTime("2018-12-22T14:00:00")  # Krakatau eruption 2018
    st = Stream()
    for station in ["GE.BBJI", "GE.MNAI", "GE.PMBI", "GE.SMRI", "GE.UGM"]:
      net, stat = station.split(".")
      st_ = client.get_waveforms(net, stat, "*", "LHZ",
                                t, t + 2 * 60 * 60)
      st_.write(f"{station}..LHZ.mseed", format='MSEED')
      st += st_
    st.write(fout, format='MSEED')
else:
  st = read(fout)

print(st)
st.plot()

- again, waveform data is returned as a Stream object
- for all custom processing workflows it does not matter if the data originates from a local file or from a web service

#### Event Metadata

The FDSN client can also be used to request event metadata:

In [ ]:
t = UTCDateTime("2018-12-22T13:50:00")
client = Client("Geofon")
catalog = client.get_events(starttime=t, endtime=t + 3600,
                            minmagnitude=2)
print(catalog)
catalog.plot();

Requests can have a wide range of constraints (see [ObsPy Documentation](http://docs.obspy.org/packages/autogen/obspy.fdsn.client.Client.get_events.html)):

- time range
- geographical (lonlat-box, circular by distance)
- depth range
- magnitude range, type
- contributing agency

#### Station Metadata

Finally, the FDSN client can be used to request station metadata. Stations can be looked up using a wide range of constraints (see [ObsPy documentation](http://docs.obspy.org/packages/autogen/obspy.fdsn.client.Client.get_stations.html)):

 * network/station code
 * time range of operation
 * geographical (lonlat-box, circular by distance)

In [ ]:
event = catalog[-1]
origin = event.origins[0]

inventory = client.get_stations(longitude=origin.longitude, latitude=origin.latitude,
                                maxradius=3.5, level="station")
print(inventory)

The **`level=...`** keyword is used to specify the level of detail in the requested inventory

- `"network"`: only return information on networks matching the criteria
- `"station"`: return information on all matching stations
- `"channel"`: return information on available channels in all stations networks matching the criteria
- `"response"`: include instrument response for all matching channels (large result data size!)

In [ ]:
inventory = client.get_stations(network="GE", station="BBJI",
                                level="station")
print(inventory)

In [ ]:
inventory = client.get_stations(network="GE", station="BBJI",
                                level="channel")
print(inventory)

For waveform requests that include instrument correction, the appropriate instrument response information can be attached to waveforms automatically:     
(Of course, for work on large datasets, the better choice is to download all station information and avoid the internal repeated webservice requests)

In [ ]:
t = UTCDateTime("2018-12-22T13:56:48")  # Tohoku
st = client.get_waveforms("GE", "BBJI", "*", "LHZ",
                          t, t + 10 * 60, attach_response=True)
st.plot();

st.remove_response()
st.plot();

All data requested using the FDSN client can be directly saved to file using the **`filename="..."`** option. The data is then stored exactly as it is served by the data center, i.e. without first parsing by ObsPy and outputting by ObsPy.

In [ ]:
client.get_events(starttime=t-100, endtime=t+3600, minmagnitude=5,
                  filename="/tmp/requested_events.xml")
client.get_stations(network="GE", station="BBJI", level="station",
                    filename="/tmp/requested_stations.xml")
client.get_waveforms("GE", "BBJI", "*", "LHZ", t, t + 10 * 60,
                     filename="/tmp/requested_waveforms.mseed")
!ls -lrt /tmp/requested*

#### FDSN Client Exercise

Use the FDSN client to assemble a waveform dataset for on event.

- search for a large earthquake (e.g. by depth or in a region of your choice, use option **`limit=5`** to keep network traffic down)

- search for stations to look at waveforms for the event. stations should..
    * be available at the time of the event
    * have a vertical 1 Hz stream ("LHZ", to not overpower our network..)
    * be in a narrow angular distance around the event (e.g. 90-91 degrees)
    * adjust your search so that only a small number of stations (e.g. 3-6) match your search criteria

- for each of these stations download data of the event, e.g. a couple of minutes before to half an hour after the event
- put all data together in one stream (put the `get_waveforms()` call in a try/except/pass block to silently skip stations that actually have no data available)
- print stream info, plot the raw data

- correct the instrument response for all stations and plot the corrected data

If you have time, assemble and plot another similar dataset (e.g. like before stations at a certain distance from a big event, or use Transportable Array data for a big event, etc.)